In [ ]:
from pathlib import Path
from collections import defaultdict
from typing import Any
from pprint import pprint
from sklearn.preprocessing import LabelEncoder

import pickle as pkl
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATASETS_PATH = Path('/data/Datasets/usg-kaggle/train/')

In [ ]:
zeroes = list((DATASETS_PATH / "0").rglob('regression*'))
ones = list((DATASETS_PATH / "1").rglob("regression*"))
len(zeroes), len(ones)

In [ ]:
def get_stats(paths: list) -> dict:
    results = defaultdict(list)
    for path in paths:
        datum = json.loads(path.read_text())
        for key, value in datum.items():
            results[key].append(value)
            
    plt.figure()
    plt.hist(results["mean"], bins=30)
    print(np.median(results["mean"]))
                
    return {
        "sd": np.mean(results["sd"]),
        "min": np.min(results["min"]),
        "max": np.max(results["max"]),
        "mean": np.mean(results["mean"])
    }

def get_mean_stats(paths: list) -> dict:
    means_stats = []
    for path in paths:
        datum = json.loads(path.read_text())
        means_stats.append(datum["mean"])
                
    return {
        "sd": np.std(means_stats),
        "min": np.min(means_stats),
        "max": np.max(means_stats),
        "mean": np.mean(means_stats)
    }

def get_unique_decimals(paths: list) -> set:
    decimals = []
    for path in paths:
        datum = json.loads(path.read_text())
        decimals.append(int(datum["mean"]))
    return set(decimals)

def get_decimals(paths: list) -> list:
    decimals = []
    for path in paths:
        datum = json.loads(path.read_text())
        decimals.append(int(datum["mean"]))
    return decimals

def get_unique_floats(paths: list) -> set:
    floats = []
    for path in paths:
        datum = json.loads(path.read_text())
        floats.append(int(datum["mean"] * 10) % 10)
    return set(floats)

In [ ]:
get_stats(zeroes), get_stats(ones)

In [ ]:
get_mean_stats(zeroes), get_mean_stats(ones)

In [ ]:
decimals = list(get_unique_decimals(zeroes + ones))
np.min(decimals), np.max(decimals)

In [ ]:
get_unique_floats(zeroes + ones)

In [ ]:
ones_decims = get_decimals(ones)
zeroes_decims = get_decimals(zeroes)
plt.hist(ones_decims, bins=61)
plt.hist(zeroes_decims, bins=61)

In [ ]:
np.max(ones_decims) - np.min(zeroes_decims)